In [1]:
main_dict = {
    'bank':[
        'BBVA BANCOMER',
        'BANORTE',
        'SANTANDER',
        'BANCO NACIONAL DE MEXICO',
        'BANREGIO',
        'BANAMEX',
        'HSBC',
        'BANCO AFIRME',
        'BANCO AZTECA',
        'BANCOPPEL',
        'SCOTIABANK',
        'BANCO REGIONAL DE MONTERREY',
        'BANCO DEL BAJIO',
    ]
}

In [2]:
# Control de datos
from typing import Dict
from pathlib import Path
from IPython.display import display

# Ingeniería de variables
from re import sub, UNICODE
from numpy import nan, array
from unicodedata import normalize
from datetime import datetime, date
from difflib import get_close_matches
from pandas import DataFrame, read_csv, to_datetime, qcut

# Modelos
from sklearn.pipeline import Pipeline
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from category_encoders.cat_boost import CatBoostEncoder

class BaseClass:
    def __init__(self) -> None:
        pass

    def clean_text(self, text: str, pattern: str="[^a-zA-Z0-9\s]", lower: bool=False) -> str: 
        '''
        Limpieza de texto
        '''
        # Reemplazar acentos: áàäâã --> a
        clean = normalize('NFD', str(text).replace('\n', ' \n ')).encode('ascii', 'ignore')
        # Omitir caracteres especiales !"#$%&/()=...
        clean = sub(pattern, ' ', clean.decode('utf-8'), flags=UNICODE)
        # Mantener sólo un espacio
        clean = sub(r'\s{2,}', ' ', clean.strip())
        # Minúsculas si el parámetro lo indica
        if lower: clean = clean.lower()
        # Si el registro estaba vacío, indicar nulo
        if clean in ('','nan'): clean = nan
        return clean

    def choose_correct(self, df: DataFrame, col: str, correct_list: list, fill_value: str='Otro', keep_nan: bool=True, replace_col: bool=True, **kwargs) -> DataFrame:
        '''
        Recibe un DataFrame y una lista de posibilidades, especificando la columna a revisar
        elige la opción que más se parezca a alguna de las posibilidades
        '''
        # Aplicar limpieza de texto a la lista de posibilidades
        correct_clean = list(map(lambda x: self.clean_text(x, lower=True), correct_list))+['nan']
        # Hacer un diccionario de posibilidades limpias y las originales recibidas
        correct_dict = dict(zip(correct_clean, correct_list+['nan']))

        # Aplicar la limpieza a la columna especificada
        df[f'{col}_correct'] = df[col].map(lambda x: self.clean_text(x,lower=True))
        # Encontrar las posibilidades más parecidas
        df[f'{col}_correct'] = df[f'{col}_correct'].map(lambda x: get_close_matches(str(x), correct_clean, **kwargs))
        # Si existen parecidas, traer la primera opción que es la más parecida
        df[f'{col}_correct'] = df[f'{col}_correct'].map(lambda x: x[0] if isinstance(x,list) and len(x)>0 else nan)
        # Regresar del texto limpio a la posibilidad original, lo no encontrado se llena con "fill_value"
        df[f'{col}_correct'] = df[f'{col}_correct'].map(correct_dict).fillna(fill_value)
        
        if keep_nan: df[f'{col}_correct'] = df[f'{col}_correct'].map(lambda x: nan if str(x)=='nan' else x)
        if replace_col: df = df.drop(col, axis=1).rename({f'{col}_correct':col}, axis=1)
        return df
        
    def two_char(self, n): 
        '''
        Función para convertir float: 1.0 --> str: '01.00'
        '''
        return str(round(n,2)).zfill(2)

    def get_bins(self, df: DataFrame, col: str, bin_dict: Dict, n_bins: int=5, replace_col: bool=False) -> DataFrame:
        # Encontrar el bin al cual el dato pertenece
        df[f'{col}_range'], bin_dict[f'{col}_bins'] = qcut(df[col], q=n_bins, retbins=True, duplicates='drop')
        # Convertirlo a texto: [1.0 - 5.0] --> '01.00 a 05.00'
        df[f'{col}_range'] = df[f'{col}_range'].map(lambda x: nan if str(x)=='nan' else self.two_char(x.left)+' to '+self.two_char(x.right))
        if replace_col: df = df.drop(col, axis=1).rename({f'{col}_range':col}, axis=1)
        return df


    def bin_distrib(self, df: DataFrame, id_col: str, col_to_count: str, bin_dict: Dict) -> DataFrame:
        df = self.get_bins(df, col_to_count, bin_dict=bin_dict, replace_col=True)
        df['n'] = 1
        df = df.pivot_table(index=id_col, columns=col_to_count, values='n', aggfunc=sum)
        aux = df.copy()
        aux['total'] = aux.sum(axis=1)
        for col in df.columns: df[col] /= aux['total']
        df.rename({x:f'{col_to_count}_{x}' for x in df.columns}, axis=1, inplace=True)
        return df


    def train_model(self, X: DataFrame, y: array, encoder=CatBoostEncoder, scaler=RobustScaler, model=LogisticRegression, **kwargs) -> tuple: 
        '''
        Escala y entrena un modelo, devuelve el score, el objeto tipo Pipeline y la relevancia de cada variable
        '''
        # Conjunto de entrenamiento y de test
        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.85, random_state=7, shuffle=True)

        # Define los pasos del flujo
        pipe_obj = Pipeline(steps=[
            ('encoder', encoder()),
            ('scaler', scaler()),
            ('model', model(**kwargs))
        ])

        # Entrena y guarda el score en test
        test_score = pipe_obj.fit(X_train,y_train).score(X_test, y_test)
        # Guarda el score en train, para revisar sobreajuste
        train_score = pipe_obj.score(X_train,y_train)

        # Imprime los scores
        print(f"Score: {'{:.2%}'.format(test_score)}\nTraining score: {'{:.2%}'.format(train_score)}")

        # Elige la forma de obtener las variables más representativas
        # Ya sea por Regresión Lineal
        try: most_important_features = pipe_obj[-1].coef_[0]
        except: 
            # O por Árbol de decisión, Bosque Aleatorio, XGBoost
            try: most_important_features = pipe_obj[-1].feature_importances_
            # De otro modo, solamente asignar un vector de 0s a este objeto
            except: most_important_features = [0]*len(X.columns)

        # Las ordena descendentemente
        coef_var = DataFrame(zip(X.columns, most_important_features)).sort_values(1, ascending=False).reset_index(drop=True)

        # Devuelve el objeto para clustering, la lista de scores tanto en train como en test y la relevancia de cada variable para el modelo 
        return pipe_obj, (test_score,train_score), coef_var


    def profiles(self, df: DataFrame, cluster_col: str='cluster', to_show: int=3) -> None: 
        '''
        Recibe el resultado del método anterior para mostrar la diferencia numérica y categórica de cada clúster para todas las variables
        '''
        prof = {}
        # Obtener el tipo de variable para cada columna
        df_coltype = df.dtypes

        # Guardar las variables numéricas
        num_cols = [x for x,y in zip(df_coltype.index, df_coltype) if y!=object or x==cluster_col]
        try: num_cols.remove(cluster_col)
        except: pass
        # Promedio de cada variable numérica según el clúster
        if len(num_cols)>0: prof['numeric'] = df.pivot_table(index=cluster_col, values=num_cols)

        # Obtener las variables categóricas
        cat_cols = [x for x in df.columns if x not in num_cols]

        # Columna auxiliar para contar registros
        df['n'] = 1
        for col in cat_cols: 
            # Cuenta de registros para cada variable categórica según el clúster
            prof[col] = df.pivot_table(index=cluster_col, columns=col, aggfunc={'n': sum})

        # Mostrar cada perfilamiento en un DataFrame con formato condicional
        for x in prof.values():
            x = x.fillna(0)
            # Con grupo en renglones, valores de clase en columna
            by_clust = x.copy()
            # Igual que la anterior, pero mostrando el % del total
            perc = x/x.sum().sum()
            # Al revés, clúster en columnas
            by_var = x.T.copy()

            # Mostrar las tres tablas, para notar distribución por clúster, distrib por valor de clase a través de clúster y el % del total
            for i, to_plot in enumerate(zip([by_clust, by_var, perc], ["{:.1f}","{:.1f}","{:.1%}"], [0,0,None])):
                # Desagrupa el tuple que se enumeró
                summary, to_format, to_axis = to_plot
                # Mostrar el número de gráficas indicado para cada tipo de variable
                if i+1 <= to_show:
                    # Aplicar el formato de número y formato condicional asignado
                    display(summary.style.format(to_format).background_gradient('Blues', axis=to_axis))








class ShoppersChurn(BaseClass):
    def __init__(self, file_name: str, main_dict: Dict=main_dict) -> None:
        self.main_dict = main_dict
        self.base_dir = Path.cwd().parent
        self.data_dir = self.base_dir.joinpath('data')
        self.file_name = file_name
        self.file_path = self.data_dir.joinpath(f'{self.file_name}.csv')
        if not self.file_path.is_file():
            print(f'There should be a file called "{self.file_name}" at:\n{self.data_dir}\n\nAdd it and try again!')


    def get_files(self, shopper_id_col: str='shopper_id') -> None:
        data = read_csv(self.file_path, low_memory=False)
        end_of_shopper_data = [x for x,y in enumerate(data.columns) if y=='end_of_shoppers_data'][0]
        self.sh = data.iloc[:,:end_of_shopper_data].drop_duplicates(shopper_id_col)
        self.df = data[[shopper_id_col]].join(data.iloc[:,end_of_shopper_data+1:])


    def clean_shopper_data(self, marital_col: str='marital_status', insurance_col: str='insurance', bank_col: str='bank', transport_col: str='transport') -> None:
        df = self.sh.copy()
        df[marital_col] = df[marital_col].map(lambda x: nan if str(x)=='nan' else x.replace(' ',''))

        aux = []
        for x in df[insurance_col]:
            if str(x)=='nan': aux.append(nan)
            else: 
                try: to_append = to_datetime(x, format=r'%d/%m/%y')
                except: 
                    try: to_append = to_datetime(x[:10], format=r'%Y-%m-%d')
                    except: 
                        try: to_append = to_datetime(x[:11], format=r'%d-%b-%Y')
                        except:
                            try: to_append = to_datetime(x[:11], format=r'%d-%b-%y')
                            except: 
                                print(f'Date: "{x}" was not converted successfully')
                                to_append = nan
                finally: aux.append(to_append)
        df[insurance_col] = aux

        df = self.choose_correct(df, bank_col, self.main_dict[bank_col], n=1, cutoff=0.7)

        df[transport_col] = df[transport_col].map(lambda x: nan if str(x)=='nan' else x.split()[0].title())
        aux = df[transport_col].value_counts(1).to_frame()
        self.main_dict[transport_col] = [x for x,y in zip(aux.index, aux[transport_col]) if y>=0.02]
        df = self.choose_correct(df, transport_col, self.main_dict[transport_col], n=1, cutoff=0.7)

        self.sh = df.copy()


    def vars_shopper(self, id_col: str='shopper_id', official_id_col: str='official_id', insurance_col: str='insurance', last_date_col: str='last_date') -> None:
        df = self.sh.set_index(id_col)

        df['birthday'] = to_datetime(df[official_id_col].str[4:10], format=r'%y%m%d')
        df['birthday'] = df['birthday'].map(lambda x: date(x.year-100, x.month, x.day) if x.year>datetime.today().year else x)
        df['age_in_days'] = (datetime.today() - df['birthday']).dt.days

        df['genre'] = df[official_id_col].str[10:11]

        df[last_date_col] = to_datetime(df[last_date_col])
        df['is_churn'] = ((datetime.today() - df[last_date_col]).dt.days//7 >= 4)*1
        df['days_for_insurance_exp'] = df[[insurance_col,last_date_col]].apply(lambda x: nan if str(x[0])=='nan' else (x[0] - x[-1]).days, axis=1)
        
        df.drop([official_id_col, 'birthday', insurance_col, last_date_col], axis=1, inplace=True)

        self.main_dict['shop_num_cols'] = df.sample(frac=0.1).describe().columns.tolist()
        self.main_dict['shop_cat_cols'] = [x for x in df.columns if x not in self.main_dict['shop_num_cols']]
        self.main_dict['shop_num_cols'].remove('is_churn')

        self.main_dict['shop_bin_dict'] = {}
        for col in self.main_dict['shop_num_cols']:
            df = self.get_bins(df, col, self.main_dict['shop_bin_dict'], replace_col=True)
            df[col] = df[col].cat.add_categories('Unknown')

        df.fillna('Unknown', inplace=True)
        self.sh = df.astype(str).copy()


    def vars_orders(self, id_col: str='shopper_id') -> None:
        df = self.df.copy()
        df['n_orders'] = 1
        agg = df.pivot_table(index=id_col, values='n_orders', aggfunc=sum)
        for col in df.columns[:-1]:
            if col!=id_col: agg = agg.join(self.bin_distrib(df, id_col, col, self.main_dict), )
        
        self.total = self.sh.join(agg.fillna('Unknown'))


    def shoppers_train_model(self) -> None:
        df = sc.total.copy()
        X = df[[x for x in df.columns if x not in ['is_churn']]].copy()
        y = df['is_churn'].values
        model, _, coefs = self.train_model(X, y)
        # self.profiles(df, cluster_col='is_churn')
        self.main_dict['shopper_model'] = model
        display(coefs)
        self.total['predict_proba'] = [x[-1] for x in model.predict_proba(X)]
        self.total.to_csv(sc.base_dir.joinpath('shopper_predict.csv'))


sc = ShoppersChurn(file_name='raw_data_MTY')
sc.get_files()
sc.clean_shopper_data()
sc.vars_shopper()
sc.vars_orders()
sc.shoppers_train_model()

Date: "0029-10-20" was not converted successfully


/Users/efraflores/Desktop/hub/shoppers_churn/venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Score: 81.60%
Training score: 82.88%


,0,1
0,seniority,0.737919
1,completed_deliveries,0.690142
2,days_elapsed_-0.0 to 0.03,0.349679
3,picking_commission_vs_total_-0.0 to 0.18,0.273571
4,days_for_insurance_exp,0.233403
...,...,...
87,days_elapsed_0.03 to 0.05,-0.166973
88,total_commission_88.35 to 113.73,-0.181329
89,pickings_vs_deliveries,-0.222406
90,completed_pickings,-0.406206
